In [1]:
import pandas as pd
import json

In [2]:
#Read the data
data = pd.read_json('Export_smallDataFrame.json')

In [3]:
data.head()

,id,categories,abstract,encoded_categories,stemmed abstract,lemma abstract,length
0,704.0003,physics-ph,evolution earthmoon system described dark matt...,10234,evolut earthmoon system describ dark matter fi...,evolution earthmoon system described dark matt...,616
1,704.0004,math,show determinant stirling cycle numbers counts...,7149,show determin stirl cycl number count unlabel ...,show determinant stirling cycle number count u...,195
2,704.0005,math,paper show compute lambda_alpha norm alphage 0...,7149,paper show comput lambda_alpha norm alphag 0 u...,paper show compute lambda_alpha norm alphage 0...,140
3,704.0010,math,partial cubes isometric subgraphs hypercubes s...,7149,partial cube isometr subgraph hypercub structu...,partial cube isometric subgraphs hypercubes st...,529
4,704.0011,math,paper present algorithm computing hecke eigens...,7149,paper present algorithm comput heck eigensyste...,paper present algorithm computing hecke eigens...,256


In [4]:
len(data)

84000

In [5]:
data.groupby(['categories']).size()

categories
cs            21000
math          21000
physics-ph    21000
stat          21000
dtype: int64

In [6]:
#splitting test and train data ahead of time
from sklearn.model_selection import train_test_split
X = data["stemmed abstract"]
y = data['encoded_categories']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
#TF IDF with stems
from sklearn.feature_extraction.text import TfidfVectorizer 
vectorizer = TfidfVectorizer(max_df = 0.8, min_df = .1)
X_train_knn = vectorizer.fit_transform(X_train)
X_test_knn = vectorizer.transform(X_test)

In [25]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=20) #algorithm = "brute", n_jobs=-1
neigh.fit(X_train_knn, y_train)
print(neigh.score(X_test_knn, y_test))  # k=3 0.6336904761904761 k=5 0.6548809523809523
# k=7 0.6626785714285715, k=8  0.6626785714285715

0.6685119047619048


In [22]:
from sklearn.ensemble import RandomForestClassifier                     
rfc = RandomForestClassifier(max_depth=18) # a lot slower if you don't set a max_depth... I didn't bother waiting
rfc.fit(X_train_knn, y_train)  
print(rfc.score(X_test_knn, y_test))   #0.7239285714285715 depth15,  0.7076785714285714 depth 10,
#0.726845238095238 depth=16

0.7313690476190476


In [12]:
from sklearn.linear_model import SGDClassifier

sgd= SGDClassifier()
sgd.fit(X_train_knn, y_train)
#mean of accuracy
print(sgd.score(X_test_knn, y_test))

0.7100595238095239


In [21]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
#tfidf_train = tfidf.fit_transform(x_train) 
#tfidf_test = tfidf.transform(x_test)
#tfidf_test_final = tfidf.transform(test['text'])



#X_train_knn = vectorizer.fit_transform(X_train)
#X_test_knn = vectorizer.transform(X_test)

# Setting up Passive Aggressive Classifier
pac = PassiveAggressiveClassifier(max_iter = 100) # with 50 get 60
# Fitting on the training set
pac.fit(X_train_knn, y_train)
# Predicting on the test set
y_pred = pac.predict(X_test_knn)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score * 100, 2)}%')

Accuracy: 66.65%
